## Import Credentials

In [0]:
%run "./Config"

In [0]:
# Mounting Directory for Yelp JSONs Intake
storageAccount = "gen10datafund2205"
storageContainer = "group1-marketing-analytics"
mount_point = "/mnt/group1/incomein"
    
    
configs2 = {"fs.azure.account.auth.type": "OAuth",
   "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
   "fs.azure.account.oauth2.client.id": clientid,
   "fs.azure.account.oauth2.client.secret": clientSecret,
   "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/d46b54b2-a652-420b-aa5a-2ef7f8fc706e/oauth2/token",
   "fs.azure.createRemoteFileSystemDuringInitialization": "true"}

try:
    dbutils.fs.unmount(mount_point)
except:
    pass


dbutils.fs.mount(
source = f"abfss://{storageContainer}@{storageAccount}.dfs.core.windows.net/", 
mount_point = mount_point, 
extra_configs = configs2)

/mnt/group1/incomein has been unmounted.
Out[1]: True

In [0]:
from pyspark.sql.functions import udf,col

In [0]:
data = sc.textFile('/mnt/group1/incomein/ACSST5Y2020.S1901_data_with_overlays_2022-04-22T121513.csv')
firstRow=data.first()
data=data.filter(lambda row:row != firstRow)


In [0]:
df = spark.read.csv(data, header = True)

In [0]:
df = df.withColumn("ZipCode", df.id.substr(10,14)) 

In [0]:
df.dtypes

Out[6]: [('id', 'string'),
 ('Geographic Area Name', 'string'),
 ('Estimate!!Households!!Total', 'string'),
 ('Margin of Error!!Households!!Total', 'string'),
 ('Estimate!!Households!!Total!!Less than $10,000', 'string'),
 ('Margin of Error!!Households!!Total!!Less than $10,000', 'string'),
 ('Estimate!!Households!!Total!!$10,000 to $14,999', 'string'),
 ('Margin of Error!!Households!!Total!!$10,000 to $14,999', 'string'),
 ('Estimate!!Households!!Total!!$15,000 to $24,999', 'string'),
 ('Margin of Error!!Households!!Total!!$15,000 to $24,999', 'string'),
 ('Estimate!!Households!!Total!!$25,000 to $34,999', 'string'),
 ('Margin of Error!!Households!!Total!!$25,000 to $34,999', 'string'),
 ('Estimate!!Households!!Total!!$35,000 to $49,999', 'string'),
 ('Margin of Error!!Households!!Total!!$35,000 to $49,999', 'string'),
 ('Estimate!!Households!!Total!!$50,000 to $74,999', 'string'),
 ('Margin of Error!!Households!!Total!!$50,000 to $74,999', 'string'),
 ('Estimate!!Households!!Total!!$75,000 to $99,999', 'string'),
 ('Margin of Error!!Households!!Total!!$75,000 to $99,999', 'string'),
 ('Estimate!!Households!!Total!!$100,000 to $149,999', 'string'),
 ('Margin of Error!!Households!!Total!!$100,000 to $149,999', 'string'),
 ('Estimate!!Households!!Total!!$150,000 to $199,999', 'string'),
 ('Margin of Error!!Households!!Total!!$150,000 to $199,999', 'string'),
 ('Estimate!!Households!!Total!!$200,000 or more', 'string'),
 ('Margin of Error!!Households!!Total!!$200,000 or more', 'string'),
 ('Estimate!!Households!!Median income (dollars)', 'string'),
 ('Margin of Error!!Households!!Median income (dollars)', 'string'),
 ('Estimate!!Households!!Mean income (dollars)', 'string'),
 ('Margin of Error!!Households!!Mean income (dollars)', 'string'),
 ('Estimate!!Households!!PERCENT ALLOCATED!!Household income in the past 12 months',
 'string'),
 ('Margin of Error!!Households!!PERCENT ALLOCATED!!Household income in the past 12 months',
 'string'),
 ('Estimate!!Households!!PERCENT ALLOCATED!!Family income in the past 12 months',
 'string'),
 ('Margin of Error!!Households!!PERCENT ALLOCATED!!Family income in the past 12 months',
 'string'),
 ('Estimate!!Households!!PERCENT ALLOCATED!!Nonfamily income in the past 12 months',
 'string'),
 ('Margin of Error!!Households!!PERCENT ALLOCATED!!Nonfamily income in the past 12 months',
 'string'),
 ('Estimate!!Families!!Total', 'string'),
 ('Margin of Error!!Families!!Total', 'string'),
 ('Estimate!!Families!!Total!!Less than $10,000', 'string'),
 ('Margin of Error!!Families!!Total!!Less than $10,000', 'string'),
 ('Estimate!!Families!!Total!!$10,000 to $14,999', 'string'),
 ('Margin of Error!!Families!!Total!!$10,000 to $14,999', 'string'),
 ('Estimate!!Families!!Total!!$15,000 to $24,999', 'string'),
 ('Margin of Error!!Families!!Total!!$15,000 to $24,999', 'string'),
 ('Estimate!!Families!!Total!!$25,000 to $34,999', 'string'),
 ('Margin of Error!!Families!!Total!!$25,000 to $34,999', 'string'),
 ('Estimate!!Families!!Total!!$35,000 to $49,999', 'string'),
 ('Margin of Error!!Families!!Total!!$35,000 to $49,999', 'string'),
 ('Estimate!!Families!!Total!!$50,000 to $74,999', 'string'),
 ('Margin of Error!!Families!!Total!!$50,000 to $74,999', 'string'),
 ('Estimate!!Families!!Total!!$75,000 to $99,999', 'string'),
 ('Margin of Error!!Families!!Total!!$75,000 to $99,999', 'string'),
 ('Estimate!!Families!!Total!!$100,000 to $149,999', 'string'),
 ('Margin of Error!!Families!!Total!!$100,000 to $149,999', 'string'),
 ('Estimate!!Families!!Total!!$150,000 to $199,999', 'string'),
 ('Margin of Error!!Families!!Total!!$150,000 to $199,999', 'string'),
 ('Estimate!!Families!!Total!!$200,000 or more', 'string'),
 ('Margin of Error!!Families!!Total!!$200,000 or more', 'string'),
 ('Estimate!!Families!!Median income (dollars)', 'string'),
 ('Margin of Error!!Families!!Median income (dollars)', 'string'),
 ('Estimate!!Families!!Mean income (dollars)', 'string'),
 ('Margin of Error!!Families!!Mean income (dol

In [0]:
#filter to keep only zipcodes and median household income
df= df.select(['ZipCode','Estimate!!Households!!Median income (dollars)'])

In [0]:
#remove dash values
df = df.filter(col('Estimate!!Households!!Median income (dollars)') != '-' )


In [0]:
from pyspark.sql.functions import isnan, when, count, col
#check for nulls in each column
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+-------+---------------------------------------------+
ZipCode|Estimate!!Households!!Median income (dollars)|
+-------+---------------------------------------------+
 0| 0|
+-------+---------------------------------------------+

In [0]:
#convert Strings to ints 

from pyspark.sql.types import IntegerType


df = df.withColumn("Estimate!!Households!!Median income (dollars)", df["Estimate!!Households!!Median income (dollars)"].cast(IntegerType()))

In [0]:
df.dtypes

Out[11]: [('ZipCode', 'string'),
 ('Estimate!!Households!!Median income (dollars)', 'int')]

In [0]:
database = "Group1-Marketing-Analytics"
user = "Group1"
password  = "D9k8n7z6g%mar!keting"
server = "gen10-data-fundamentals-22-05-sql-server.database.windows.net"

In [0]:
df = df.withColumnRenamed('Estimate!!Households!!Median income (dollars)','MedianIncome')

In [0]:
df.write.format('json').mode('overwrite').save(
    mount_point+'/CensusIncome_cleaned.json')